In [3]:
# Load the LunaSolve package
from luna_sdk import LunaSolve
from luna_sdk.schemas.optimization_formats.bqm import BQMSchema
from luna_sdk.schemas.enums.timeframe import TimeframeEnum

import neal
import numpy as np
import time


from data.sp_data import SPData
from models import SPQuboBinary
from evaluation.evaluation import SPEvaluation
from plotting.sp_plot import SPPlot
import matplotlib.pyplot as plt

import numpy as np

import os

# Set up your encryption key here
encryption_key = "3U6MhEmbVkC1kZW7OP9tNKf65Xz5wVtE3J3hrylf8Dc="

# Initialize LunaSolve with your API key
ls = LunaSolve(api_key="91d60a61f6b44964881f43f4e7673124")



### List of issues in classical simulation algorithms :

Unavailable solver :
- ("QAOA", "ibm"): Access to IBM is currently not available on Luna.
- ("VQE", "ibm"): Access to IBM is currently not available on Luna.
- ("BF", ): Error message: Solver unavailable
- ("SAGA_PL","dwave"): Error message: Solver unavailable
- ("SAGA_PW","dwave"): Error message: Solver unavailable

Solver with docs issue :
- ("PT", "dwave"): invalid param (rtol)
- ("QLSA", "dwave"): invalid param (rtol)
- ("TS","dwave"): invalid params (num_reads, tenure, timeout, initial_state_generator)
- ("DS","dwave"): invalid param (rtol)
- ("QLTS","dwave"): invalid param (rtol)

In [14]:
def create_jobs_for_one_instance(qubo_matrix, instance_id, ls):
    """
    This function create luna job for each possible optimization techniques for one qubo instance.
    
    Return a list of the jobs ids.
    """
    print("Start creating job")
    
    #Create jobs for each method (not very readable code, sorry for that)
    job_ids = []
    # Example of using SCIP using the Luna backend in LunaSolve
    SCIP = ls.solution.create(
        optimization_id=instance_id,
        solver_name="SCIP",
        provider="zib",
        solver_parameters={
        },
        qpu_tokens=None,
        encryption_key=encryption_key
    )
    job_ids.append(SCIP.id)
    # Example of using SA using the Luna backend in LunaSolve
    SA = ls.solution.create(
        optimization_id=instance_id,
        solver_name="SA",
        provider="dwave",
        solver_parameters={
            "num_reads": None,
            "num_sweeps": 1000,
            "beta_range": None,
            "beta_schedule_type": "geometric",
            "initial_states_generator": "random",
            "num_sweeps_per_beta": 1,
            "seed": None,
            "beta_schedule": None,
            "initial_states": None,
            "randomize_order": False,
            "proposal_acceptance_criteria": "Metropolis",
        },
        qpu_tokens=None,
        encryption_key=encryption_key
    )
    job_ids.append(SA.id)
    # Example of using PA using the Luna backend in LunaSolve
    PA = ls.solution.create(
        optimization_id=instance_id,
        solver_name="PA",
        provider="dwave",
        solver_parameters={
            "fixed_temperature_sampler": {
                "num_sweeps": 10000,
                "num_reads": None,
            },
            "max_iter": 20,
            "max_time": 2,
        },
        qpu_tokens=None,
        encryption_key=encryption_key
    )
    job_ids.append(PA.id)
    
    # Example of using PT using the Luna backend in LunaSolve
    PT = ls.solution.create(
        optimization_id=instance_id,
        solver_name="PT",
        provider="dwave",
        solver_parameters={
            "n_replicas": 2,
            "random_swaps_factor": 1,
            "fixed_temperature_sampler": {
                "num_sweeps": 10000,
                "num_reads": None,
            },
            "cpu_count_multiplier": 5,
            "loop": {
                "max_iter": 100,
                "max_time": 5,
                "convergence": 3,
                "target": None,
                #"rtol": 1e-05,
                "atol": 1e-08,
            },
        },
        qpu_tokens=None,
        encryption_key=encryption_key
    )
    job_ids.append(PT.id)
    
    # Example of using RRSA using the Luna backend in LunaSolve
    RRSA = ls.solution.create(
        optimization_id=instance_id,
        solver_name="RRSA",
        provider="dwave",
        solver_parameters={
            "simulated_annealing": {
                "num_reads": None,
                "num_sweeps": None,
                "beta_range": None,
                "beta_schedule_type": "geometric",
                "initial_states_generator": "random",
                "num_sweeps_per_beta": 1,
                "beta_schedule": None,
                "randomize_order": False,
                "proposal_acceptance_criteria": "Metropolis",
            },
            "num_reads_per_iter": None,
            "initial_states": None,
            "timeout": 5.0,
            "max_iter": 10,
            "target": None,
        },
        qpu_tokens=None,
        encryption_key=encryption_key
    )
    job_ids.append(RRSA.id)
    
    # Example of using QLSA using the Luna backend in LunaSolve
    QLSA = ls.solution.create(
        optimization_id=instance_id,
        solver_name="QLSA",
        provider="dwave",
        solver_parameters={
            "qbsolv_like": {
                "decomposer_size": 50,
                "rolling": True,
                "rolling_history": 0.15,
                "cpu_count_multiplier": 1,
                "loop": {
                    "max_iter": 100,
                    "max_time": 5,
                    "convergence": 3,
                    "target": None,
                    #"rtol": 1e-05,
                    "atol": 1e-08,
                },
            },
            "simulated_annealing": {
                "num_reads": None,
                "num_sweeps": 1000,
                "beta_range": None,
                "beta_schedule_type": "geometric",
                "initial_states_generator": "random",
            },
        },
        qpu_tokens=None,
        encryption_key=encryption_key
    )
    job_ids.append(QLSA.id)
    
        # Example of using SAGA+ using the Luna backend in LunaSolve
    SAGA_plus = ls.solution.create(
        optimization_id=instance_id,
        solver_name="SAGA+",
        provider="dwave",
        solver_parameters={
            "p_size": 20,
            "p_inc_num": 5,
            "p_max": 160,
            "pct_random_states": 0.25,
            "mut_rate": 0.5,
            "rec_rate": 1,
            "rec_method": "random_crossover",
            "select_method": "simple",
            "target": None,
            "atol": 0.0,
            "rtol": 0.0,
            "timeout": 60.0,
            "max_iter": 100,
            "num_sweeps": 10,
            "num_sweeps_inc_factor": 1.2,
            "num_sweeps_inc_max": 7000,
            "beta_range_type": "default",
            "beta_range": None,
        },
        qpu_tokens=None,
        encryption_key=encryption_key
    )
    
    job_ids.append(SAGA_plus.id)

    # Example of using TS using the Luna backend in LunaSolve
    TS = ls.solution.create(
        optimization_id=instance_id,
        solver_name="TS",
        provider="dwave",
        solver_parameters={
            #"num_reads": None,
            #"tenure": None,
            #"timeout": 100,
            #"initial_states_generator": "random",
            "initial_states": None,
            "seed": None,
            "num_restarts": 1000000,
            "energy_threshold": None,
            "coefficient_z_first": None,
            "coefficient_z_restart": None,
            "lower_bound_z": None,
        },
        qpu_tokens=None,
        encryption_key=encryption_key
    )
    job_ids.append(TS.id)
    
    # Example of using DS using the Luna backend in LunaSolve
    DS = ls.solution.create(
        optimization_id=instance_id,
        solver_name="DS",
        provider="dwave",
        solver_parameters={
            "decomposer": {
                "size": 10,
                "min_gain": None,
                "rolling": True,
                "rolling_history": 1.0,
                "silent_rewind": True,
                "traversal": "energy",
            },
            "tabu_antithesis": {
                "num_reads": None,
                "tenure": None,
                "timeout": 100,
                "initial_states_generator": "random",
            },
            "tabu_synthesis": {
                "num_reads": None,
                "tenure": None,
                "timeout": 100,
                "initial_states_generator": "random",
            },
            "loop": {
                "max_iter": 100,
                "max_time": 5,
                "convergence": 3,
                "target": None,
                #"rtol": 1e-05,
                "atol": 1e-08,
            },
            "max_tries": 100,
        },
        qpu_tokens=None,
        encryption_key=encryption_key
    )
    job_ids.append(DS.id)
    
    # Example of using QLTS using the Luna backend in LunaSolve
    QLTS = ls.solution.create(
        optimization_id=instance_id,
        solver_name="QLTS",
        provider="dwave",
        solver_parameters={
            "qbsolv_like": {
                "decomposer_size": 50,
                "rolling": True,
                "rolling_history": 0.15,
                "cpu_count_multiplier": 1,
                "loop": {
                    "max_iter": 100,
                    "max_time": 5,
                    "convergence": 3,
                    "target": None,
                    #"rtol": 1e-05,
                    "atol": 1e-08,
                },
            },
            "tabu_search": {
                "num_reads": None,
                "tenure": None,
                "timeout": 100,
                "initial_states_generator": "random",
            },
        },
        qpu_tokens=None,
        encryption_key=encryption_key
    )
    job_ids.append(QLTS.id)
    
    
    return job_ids

def collect_result_for_one_instance(job_ids, qubo_model_bin, ls):
    """
    Collect result for each job in the @param job_ids; 
    
    Return the result and a list of ids of unfinished jobs.
    """
    unfinished = []
    results = {}
    for id in job_ids:
        print("____")
       
        # After the execution of your algorithm has been finished, retrieve your solution
        r = ls.solution.get(id)
        print("Collect solution for " + r.solver)
        print(r)
        if(r.status == "DONE"):
            print("job "+ r.solver + " completed")
            out_acc = []
            for res in r.results:
                translated_dict = {int(key[1:]): 1 if value == 1.0 else 0 for key, value in res.sample.items()}
                ##Call function to compute solution !
                solution = qubo_model_bin.inverter_matrix(translated_dict)
                out = {
                        "solution": solution,
                        "energy": 0,
                        "runtime": r.runtime.total,
                        "feasible": res.feasible,
                        "info": None,
                      }
                out_acc.append(out)
            #feasible_results = filter(lambda r: r.feasible, r.results)
            #if feasible_results:
            #    lowest_feasible_result = min(feasible_results, key=lambda r: r.obj_value)
            #    print("best feasible solution has objective function: " + str(lowest_feasible_result.obj_value))
            #    print(r.runtime)
            results[r.solver]= out_acc
            #else:
            #    print("No feasible solution found")
            #    results[id] = 2**31 - 1
        else:
            print("job has not finished")
            unfinished.append(id)
    return results, unfinished

def clear_jobs_list(ls):
    """
    Delete all existing Luna's code for this luna instance.
    """
    sol = ls.solution.get_all(
            timeframe=TimeframeEnum.this_week, #Timeframe when solution was created
            limit=1000,  # Limit of solutions returned, default is 50.
            offset=0, #Pagination
    )
    i=0
    for x in sol:
        i=i+1
        ls.solution.delete(solution_id=x.id)
        print("removed solution " + x.id + " " + str(i) + "/" + str(len(sol)))
    n_sol = sol = ls.solution.get_all(
            timeframe=TimeframeEnum.this_week, #Timeframe when solution was created
            limit=1000,  # Limit of solutions returned, default is 50.
            offset=0, # Pagination
    )
    if(len(n_sol)==0):
        return True
    else:
        return False
    

In [15]:

def evaluate_one_instance(ls, qubo_model_bin, waiting_time, opt_name):
    qubo_matrix = qubo_model_bin.model
    # Upload your QUBO to LunaSolve
    optimization = ls.optimization.create_from_qubo(name=opt_name, matrix=qubo_matrix)
    instance_size = len(qubo_matrix)
    clear_jobs_list(ls)
    ids = create_jobs_for_one_instance(qubo_matrix, optimization.id, ls)
    time.sleep(waiting_time)
    results, unfinished = collect_result_for_one_instance(ids, qubo_model_bin, ls)
    if unfinished:
        print("some job isn't finished")
    return instance_size, results


In [16]:


def eval_for_one_intance(out_dict, data):
    obj_function = {}
    violations_s = {}
    runtimes = {}
    for k, v in out_dict.items():
        best_of = -1
        for ev in v:
            if(ev["feasible"]):
                evaluation = SPEvaluation(data, ev['solution'])
                #print(f"solution clean: {evaluation.solution}")
                for constraint, violations in evaluation.check_solution().items():
                    if len(violations) > 0:
                        print(f"constraint {constraint} was violated {len(violations)} times")
                    if (best_of == -1 or best_of>evaluation.get_objective()):
                        obj_function[k] = evaluation.get_objective()
                        violations_s[k] = len(violations)
                        runtimes[k] = ev["runtime"]
            else:
                print("solution isn't feasible")
    
    return obj_function, violations_s, runtimes

def create_plots(instances, ls):
    obj_function_final = {}
    violations_s_final = {}
    runtimes_final = {}
    i = 0
    for data in instances:
        i=i+1
        print("run " + str(i) + "/10")
        qubo_model_bin = SPQuboBinary(data)
        matrix = qubo_model_bin.model
        print(len(matrix))
        instance_size, out_dict = evaluate_one_instance(ls, qubo_model_bin, 100, "foo")
        of, vio, rt = eval_for_one_intance(out_dict, data)
        obj_function_final[instance_size] = of
        violations_s_final[instance_size] = vio
        runtimes_final[instance_size] = rt
    return obj_function_final, violations_s_final, runtimes_final

In [17]:
instances = []
params = {"lidar_density": 0.1, "street_point_density": 0.1}
data = SPData().create_problem_from_glb_file(**params)
instances.append(data)

print(instances)
ob_v, vio, rt = create_plots(instances, ls)

run 1/10
142
removed solution 674c10e472cca96fa1ddbd14 1/10
removed solution 674c10e572cca96fa1ddbd16 2/10
removed solution 674c10e572cca96fa1ddbd18 3/10
removed solution 674c10e672cca96fa1ddbd1a 4/10
removed solution 674c10e672cca96fa1ddbd1c 5/10
removed solution 674c10e772cca96fa1ddbd1e 6/10
removed solution 674c10e772cca96fa1ddbd20 7/10
removed solution 674c10e872cca96fa1ddbd22 8/10
removed solution 674c10f572cca96fa1ddbd24 9/10
removed solution 674c10f672cca96fa1ddbd26 10/10
Start creating job
____
Collect solution for SCIP
--------------------------------------------------------------------------------
META DATA:
--------------------------------------------------------------------------------
id: 674c114c8f045443f00372c6
name: None
created_date: 2024-12-01 07:33:32 (CET)
created_by: lenny.delzio@epfl.ch
modified_date: 2024-12-01 07:33:36 (CET)
modified_by: None
error_message: None
solver_job_info: None
params: {}
runtime: None
sense: None
solver: SCIP
provider: zib
status: StatusE

Collect solution for SAGA+
--------------------------------------------------------------------------------
META DATA:
--------------------------------------------------------------------------------
id: 674c114fe16c143066a93d2a
name: None
created_date: 2024-12-01 07:33:35 (CET)
created_by: lenny.delzio@epfl.ch
modified_date: 2024-12-01 07:33:35 (CET)
modified_by: None
error_message: None
solver_job_info: None
params:
    p_size: 20
    p_inc_num: 5
    p_max: 160
    pct_random_states: 0.25
    mut_rate: 0.5
    rec_rate: 1
    rec_method: random_crossover
    select_method: simple
    target: None
    atol: 0.0
    rtol: 0.0
    timeout: 60.0
    max_iter: 100
    num_sweeps: 10
    num_sweeps_inc_factor: 1.2
    num_sweeps_inc_max: 7000
    beta_range_type: default
    beta_range: None
runtime: None
sense: None
solver: SAGA+
provider: dwave
status: StatusEnum.IN_PROGRESS
optimization:
    id: 674c11458f045443f00372c5
    name: foo
    created_date: 2024-12-01 07:33:25 (CET)
    crea

<Figure size 640x480 with 0 Axes>

In [ ]:
def plot_data(data, y_title, title, log_y = False):
    # Extract x values and all unique keys for y values
    x_values = list(data.keys())
    x_values2 = [1, 1/2, 1/3, 1/4, 1/5, 1/6, 1/7, 1/8, 1/9]
    print(data)
    y_labels = list(next(iter(data.values())).keys())

    # Plot each y_label
    plt.figure(figsize=(10, 6))
    for label in y_labels:
        y_values = [data[x].get(label, 0) for x in x_values]  # Assign 0 if the key is missing
        
        plt.plot(x_values2, y_values, marker='o', label=label)
    
    
    plt.xlabel('Street points density')
    plt.ylabel(y_title)
    if(log_y):
        plt.yscale('log')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.savefig(title +".png")
    
plot_data(ob_v, "objective value", "evolution of objective for glb, lidar density=0.2")
plot_data(vio, "number of violation", "evolution of the number of violation for glb, lidar density =0.2")
plot_data(rt, "runtime", "evolution of runtime value  for glb, lidar density=0.2", log_y=True)